In [1]:
import pandas as pd
import numpy as np
from itertools import product
from tqdm import tqdm
from tools import *

In [2]:
df= pd.DataFrame({
    "N_candidat": ["0001", "0002", "0003", "0004", "0005"],
    "S1":[15, 11, 14, 16, 11],
    "S2": [13, 5, 12, 15, 11],
    "S3":[16, 8, 11, 15, 11],
    "S4": [17, 12, 15, 18, 11],
    "S5": [13, 10, 13, 15, 11],

    "Logique": [1, 1, 1, 0, 0],
    "BDD": [1, 1, 1, 0, 0],
    "PS": [1, 1, 1, 1, 0],
    "Moyenne": [2, 0, 1, 2, 0],
    "Eligible": [True, True, True, False, False],
    "Decision": ['A','R','L','R','R']
})
V = Vars(df)
display(V, df)

['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS', 'Moyenne', 'Eligible']

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


# ACi
## AC1

In [3]:
#ligne existe, la décision est bonne et toutes les variables de X ont les bonnes valeurs aussi
def AC1(N, X, decision, df=df):
    c = df[df.N_candidat == N].copy()
    return len(c) == 1 and  c.Decision.iloc[0] == decision and all((str(c.loc[:, [a]].iloc[0, 0]) == str(X[a])) for a in X) 

def test_AC1():
    display(df)
    # Cas de succès
    assert AC1("0001", {"Logique": 1}, 'A')
    assert AC1("0001", {"S1":15, "BDD":1, "Moyenne":2, 'Eligible': True}, 'A')
    assert AC1("0004", {"S2":15, "BDD":0}, 'R')

    # Cas d'échec
    assert not AC1("xxxx", {"S1":15, "BDD":1, "Moyenne":2, 'Eligible': True}, 'A')
    display("AC1 est bien correcte.")

test_AC1()

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


'AC1 est bien correcte.'

## AC2

In [4]:
#TODO: faire la version incrémentale de la AC2
def AC2(N_candidat, X, decision, df=df):
    candidat=df[df.N_candidat==N_candidat].copy()
    if (len(candidat)==0):
        return False
    vals=candidat.iloc[0].to_dict()
    X_var=list(X.keys())
    
    #W=list(set(candidat.columns)-set(X_var)) #optionnel

    variable_domains = {
        "S1": range(0, 21),
        "S2": range(0, 21),
        "S3": range(0, 21),
        "S4": range(0, 21),
        "S5": range(0, 21),
        "BDD": [0, 1],
        "Logique": [0, 1],
        "PS": [0, 1],
        "Moyenne":[0,1,2],
        "Eligible":[True,False]
    }
    def generate_combinations(X, variable_domains):
        # Générer toutes les combinaisons possibles de valeurs pour les clés de X_var
        combinations = product(*[variable_domains[var] for var in X])

        # Exclure la combinaison identique à X
        return [combination for combination in combinations if combination != tuple(X.values())]

    combinations = generate_combinations(X, variable_domains)
    
    # Vérifier chaque combinaison
    for combination in combinations:
        x_prime = dict(zip(X_var, combination))
        for k in x_prime.keys():
            vals[k]=x_prime[k]
        m,e,d=med(**{k: v for k, v in vals.items() if k in ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']})

        if ('Eligible' in x_prime and x_prime['Eligible']!=e) or ('Moyenne' in x_prime and x_prime['Moyenne']!=m) :
            continue
        if d!=decision:
            #print(f"AC2 est vérifié pour une affectation x_prime: {x_prime}")
            return True
    return False

def test_AC2():
	display(df)
	# Cas de succès
	assert AC2("0001", {"S1": 15},'A')
	assert AC2("0002", {"S1": 13, "S5": 11}, 'R')
	assert AC2("0003", {"Logique": 1, "S5": 11, "S2":8},'L')

	# Cas d'échec
	assert not AC2("0004", {"S5": 11, "BDD": 0},'R')
	assert not AC2("0005", {"S1": 13, "BDD": 0}, 'R')
	print("AC2 est bien correcte")

test_AC2()



,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


AC2 est bien correcte


## AC3:

In [5]:
from itertools import combinations

def AC3(N, X, decision ,df=df):
    keys = list(X.keys())
    subsets = [dict(zip(combination, (X[key] for key in combination))) for r in range(1, len(keys)) for combination in combinations(keys, r)]
    return all(not AC1(N, subset, decision, df) or not AC2(N, subset, decision) for subset in subsets)

def test_AC3():
	display(df)
	# Cas de succès
	assert AC3("0001", {"Logique": 1}, 'A')
	assert AC3("0004", {"Logique": 0, "BDD": 0}, 'R')
	assert AC3("0004", {"Logique": 0, "S1": 16}, 'R')

	# Cas d'échec
	assert not AC3("0001", {"Logique": 1, "PS": 1}, 'A')
	assert not AC3("0004", {"Logique": 0,  "BDD": 0,"S1": 16}, 'R')
	print("AC3 est bien correcte")

test_AC3()


,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


AC3 est bien correcte


In [6]:
def AC(N, X, decision, df=df, afficher = False):
  res1=AC1(N, X, decision, df)
  res2=AC2(N, X, decision, df)
  res3=AC3(N, X, decision, df)
  if afficher:
      print(f"AC1={res1} et AC2={res2} et AC3={res3}")
  return res1, res2, res3

# BCi
## BC1 et BC2

In [7]:
def BC12(N, X, decision, i=1, df=df):
	assert i==1 or i==2
	supersets = generate_supersets(N, X[i-1])
	for superset in tqdm(supersets):
		if all(AC(N, superset, decision[i-1], df)):
			return True
	return False

def BC12_beta(N, X, decision, i=1, df=df):
    #Les performances ne sont pas très bonne, en terme de complexité. Nous proposons une version améliorée.
	assert i==1 or i==2
	supersets = generate_supersets(N, X[i-1], V, df)
	for superset in tqdm(supersets):
		if not AC1(N, superset, decision[i-1], df):
			return False
		elif not AC2(N, superset, decision[i-1], df):
			continue
		elif not AC3(N, superset, decision[i-1], df):
			return False
		return True
	return False

def test_BC12(f=BC12_beta, only_echec = True):
	display(df)
	# Cas de succès
	if not only_echec:
		assert f("0001",({"S2": 13}, _), ('A', _))
		assert f("0001", ({"Logique": 1}, _), ('A', _)) 
		assert f("0005",({"Logique": 0, "BDD": 0, "PS": 0}, _), ('R', _))

	# Cas d'échec
	assert not f("0001",({"Logique": 1, "BDD": 1}, _), ('A', _)) #AC3
	assert not f("0004",({"Logique": 1, "BDD": 1, "S1": 16}, _), ('R', _)) #AC1 n'est pas vérifié
	assert not f("0004", ({"Logique": 0, "BDD": 0, "S1": 16}, _), ('R', _)) # car S1 n'appartent pas à la actual cause
	print("Les tests pour BC1 sont terminés.")
 
test_BC12(only_echec=False)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


  0%|          | 0/1024 [00:00<?, ?it/s]

Les tests pour BC1 sont terminés.


In [8]:
######################### CE QUE (RACHA) PROPOSE
def BC1_2_beta(N, X, decision, df=df):
	# verifier que X peut etre une cause partielle
	ac1, ac2, ac3 = AC(N, X, decision, df)
	if not ac1:
		return False
	if ac2 and not ac3: # AC1 et AC2 sont vérifiés et AC3 ne l'est pas --> il existe un sous enesemble de X qui verifie une actual cause donc X ne peut pas etre une partial cause 
		return False	
	if ac2 and ac3: # si X est une actual cause alors c'est une partial cause
		return True 
	supersets = generate_supersets(N, X, V, df)[1:] #////////////// ajouter [1:]
	print("Numéro candidat : ",N)
	for superset in tqdm(supersets):
		#print("\n",superset)
		ac1, ac2, ac3 = AC(N, superset, decision, df)
		if not ac1:
			continue
		if  ac2 and ac3:
			print(f"La actual cause est: {superset} \n")
			return True 
		
	return False

def test_BC1(f=BC1_2_beta, only_echec = True):
	display(df)
	# Cas de succès
	if not only_echec:
		assert f("0001", {"S2": 13}, 'A')
		assert f("0001", {"Logique": 1}, 'A') 
		assert f("0005", {"Logique": 0, "BDD": 0, "PS": 0}, 'R')

	# Cas d'échec
	assert not f("0001", {"Logique": 1, "BDD": 1}, 'A') #AC3
	assert not f("0004", {"Logique": 1, "BDD": 1, "S1": 16}, 'R') #AC1 n'est pas vérifié
	assert not f("0004", {"Logique": 0, "BDD": 0, "S1": 16}, 'R') # car S1 n'appartent pas à la actual cause
	print("Les tests pour BC1 sont terminés.")
 
test_BC1(only_echec=False)

# commented:
""" def BC1_2_beta(N, X, decision, df=df):
	supersets = generate_supersets(N, X, V, df)[1:] #////////////// ajouter [1:]

	# verifier que X peut etre une cause partielle
	ac1, ac2, ac3 = AC(N, X, decision, df)
	if not ac1:
		return False
	if ac2 and not ac3: # AC1 et AC2 sont vérifiés et AC3 ne l'est pas --> il existe un sous enesemble de X qui verifie une actual cause donc X ne peut pas etre une partial cause 
		return False	
		
	print(N)
	for superset in tqdm(supersets):
		#print("\n",superset)
		ac1, ac2, ac3 = AC(N, superset, decision, df)
		if not ac1:
			continue
		if  ac2 and ac3:
			print(f"La actual cause est: {superset} \n")
			return True 
	print("fin------------")	
	return False
 """

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


Numéro candidat :  0005


  0%|          | 0/1023 [00:00<?, ?it/s]

  1%|          | 7/1023 [00:00<00:15, 66.04it/s]

La actual cause est: {'Logique': 0, 'BDD': 0, 'PS': 0, 'S1': 11} 

Les tests pour BC1 sont terminés.


' def BC1_2_beta(N, X, decision, df=df):\n\tsupersets = generate_supersets(N, X, V, df)[1:] #////////////// ajouter [1:]\n\n\t# verifier que X peut etre une cause partielle\n\tac1, ac2, ac3 = AC(N, X, decision, df)\n\tif not ac1:\n\t\treturn False\n\tif ac2 and not ac3: # AC1 et AC2 sont vérifiés et AC3 ne l\'est pas --> il existe un sous enesemble de X qui verifie une actual cause donc X ne peut pas etre une partial cause \n\t\treturn False\t\n\t\t\n\tprint(N)\n\tfor superset in tqdm(supersets):\n\t\t#print("\n",superset)\n\t\tac1, ac2, ac3 = AC(N, superset, decision, df)\n\t\tif not ac1:\n\t\t\tcontinue\n\t\tif  ac2 and ac3:\n\t\t\tprint(f"La actual cause est: {superset} \n")\n\t\t\treturn True \n\tprint("fin------------")\t\n\treturn False\n '

## BC3

In [12]:
def BC3(X, df=None):
    X1, X2 = X
    
    # Check si les deux X1 et X2 contiennent les memes variables
    if set(X1.keys()) != set(X2.keys()):
        return False
    
    # Check que l'intersection des deux affectations est vide
    for key in X1.keys():
        if X1[key] == X2[key]:
            return False
    
    return True

def test_BC3():
    # Cas de succès
    assert BC3(({"S2":12, "BDD":0}, {"S2":11, "BDD":1}))

    # Cas d'échec
    assert not BC3(({"S1":15, "BDD":1, "Moyenne":2, 'Eligible': False}, {"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))
    assert not BC3(({"S1":15, "BDD":1, 'Eligible': False}, {"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))

    print("BC3 est bien correcte.")

test_BC3()


BC3 est bien correcte.


## BC4 

In [13]:
def BC4(N_candidats, X, decisions, df=df):
	def BCi1_3(N_candidats, X, decisions, df=df):
		return BC1_2_beta(N_candidats[0], X[0], decisions[0], df) and BC1_2_beta(N_candidats[1], X[1], decisions[1], df) and BC3(X, df)

	# S'assurer que X verifie tous les Bci1-3
	if not BCi1_3(N_candidats, X, decisions, df):
		return False
	
	candidat1=df[df.N_candidat==N_candidats[0]].copy()
	candidat2=df[df.N_candidat==N_candidats[1]].copy()
	X1=X[0]
	X2=X[1]
	X_var1=list(X1.keys())
	X_var2=list(X2.keys())
	
	V_var=list(candidat1.columns)[1:-1] # Prendre toutes les variables qui décrivent un candidat1 sauf N_candidat et Décision
	
	if X_var1 != X_var2 : # les deux candidats doivent etre testés sur les memes attributs (X)
		return False
	
	X_var=X_var1
	V_minus_X_var=[var for var in V_var if not var in X_var]

	# print(V_minus_X_var)
	
	# Trouver toutes les combinaisons possibles de longueur 1 à la longueur de la liste
	all_combinaisons_var = []

	for nb_var in range(1, len(V_minus_X_var) + 1):
			all_combinaisons_var.extend(combinations(V_minus_X_var, nb_var))
	#print(all_combinaisons_var)

	for var in tqdm(all_combinaisons_var):
		# Remise à l'état initial des sur-ensemble de X1 et X2 
		X1_eval=X1.copy()
		X2_eval=X2.copy()
		for v in var:
			X1_eval.update({v: candidat1.iloc[0, candidat1.columns.get_loc(v)]})
			X2_eval.update({v: candidat2.iloc[0, candidat2.columns.get_loc(v)]})
		X_eval=(X1_eval,X2_eval)
		#print(X_eval)
		if (BCi1_3(N_candidats, X_eval, decisions, df)):
			return False # Retourner False car on aurait trouvé un superset qui verifie tous les BC1-3
		
		

	return True
def test_BC4():
    # Cas de succès
    display(df)
    assert BC4(("0001","0004"), ({"Logique":"1"}, {"Logique":"0"}), ('A', 'R'))
    
    # Cas d'échec
    
    print("BC4 est bien correcte.")

test_BC4()

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


Numéro candidat :  0004


  1%|          | 11/1023 [00:00<00:04, 220.00it/s]


La actual cause est: {'Logique': '0', 'BDD': 0} 



100%|██████████| 511/511 [08:56<00:00,  1.05s/it] 

BC4 est bien correcte.
